In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/blog-authorship-corpus/blogtext.csv


In [6]:
df = pd.read_csv("/kaggle/input/blog-authorship-corpus/blogtext.csv")

In [7]:
df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [8]:
df.shape
df = df[ : 1000]

In [9]:
corp = df['text']

In [10]:
print(df.info())
df['age'] = df['age'].astype('object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1000 non-null   int64 
 1   gender  1000 non-null   object
 2   age     1000 non-null   int64 
 3   topic   1000 non-null   object
 4   sign    1000 non-null   object
 5   date    1000 non-null   object
 6   text    1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB
None


In [11]:
corp =corp[:1000]
import re
import nltk
from nltk.stem import WordNetLemmatizer  
lemmatizer = WordNetLemmatizer() 
from nltk.corpus import stopwords
def corp_preprocessing(corp):
    global stopwords
    corp.apply(str)
    corp = corp.apply(lambda x: re.sub(r'[^A-Za-z]+',' ',x))
    corp = corp.apply(lambda x: x.lower())
    corp = corp.apply(lambda x: x.split())
    #corp = corp.apply(lambda x: x.strip())
    stopwords = set(stopwords.words('english'))
    
    #corp = [words for words in corp if words not in stopwords]
    for words in corp:
        #print(word)
        words = [word for word in words if word not in stopwords]
        words = [lemmatizer.lemmatize(word) for word in words]
        #for word in words:
         #   if word not in stopwords:
          #      l.append(word)
    #corp = corp.apply(lambda x: lemmatizer.lemmatize(x))
    print(corp)
    
    #print(l)
    corp = corp.apply(lambda x: ' '.join([words for words in x]))
    #for words in corp:
        #print(i)
       # words =  ' '.join([word for word in words])
       # print(words)
        #corp = ' '.join([str(j) for j in i]) 
    
    #corp = [word for word in corp if word not in stopwords]
    return corp
corp = corp_preprocessing(corp)    

0      [info, has, been, found, pages, and, mb, of, p...
1      [these, are, the, team, members, drewes, van, ...
2      [in, het, kader, van, kernfusie, op, aarde, ma...
3                                     [testing, testing]
4      [thanks, to, yahoo, s, toolbar, i, can, now, c...
                             ...                        
995    [urllink, me, a, doggy, bed, need, i, say, mor...
996    [ah, yes, the, summer, olympics, are, now, hal...
997    [current, music, could, it, be, magic, donna, ...
998    [hey, this, one, of, the, many, things, i, hav...
999    [well, today, is, going, okay, i, have, applie...
Name: text, Length: 1000, dtype: object


In [12]:
corp[9]
corp[14]

'urllink here it is the superfantastic phonebox today is a great day our lovely officegirl jung mi please no letters that s what secretaries are called here went to lg twins building and got me a new phone i really needed one on account of i my old one was all in korean which was fine because i memorized where to go to do things on it and ii the screen kept going blank because i produce copious amounts of sweat when i inline skate and was dumb enough to be on the phone at the same time well after screens at won a pop i decided it s time to get a new one but i may end up breaking this one too it s soooooo cooooool it has a little camera which i used to take a picture of the box of course it took me a while to figure out that i couldn t use it to take a pic of the actual phone i also took a pic of our lovely officegirl and then inputted her name and number as well then ordered up caller id something i didn t have before much to my chagrin then i asked her to call me to test it out her mu

In [13]:
#We have to predict gender age sign and topics. This problem falls under multilabel problem.
#y = df[['gender', 'age', 'topic', 'sign']]
df['y'] = df[df.columns[1:5]].apply(lambda x: ','.join(x.astype(str)),axis=1)

In [14]:
df['x'] = corp

In [15]:
df = df[['x','y']]

In [16]:
df.head()

,x,y
0,info has been found pages and mb of pdf files ...,"male,15,Student,Leo"
1,these are the team members drewes van der laag...,"male,15,Student,Leo"
2,in het kader van kernfusie op aarde maak je ei...,"male,15,Student,Leo"
3,testing testing,"male,15,Student,Leo"
4,thanks to yahoo s toolbar i can now capture th...,"male,33,InvestmentBanking,Aquarius"


In [17]:
df.isnull().sum()
#df.dropna(inplace = True)
# why its showing all x has null value but as above x has values??

x    0
y    0
dtype: int64

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range =(1,3),binary = True)
"""ngram_rangetuple (min_n, max_n), default=(1, 1)
The lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted. All values 
of n such such that min_n <= n <= max_n will be used. For example an ngram_range of (1, 1) means only unigrams, (1, 2) means 
unigrams and bigrams, and (2, 2) means only bigrams. Only applies if analyzer is not callable."""

'ngram_rangetuple (min_n, max_n), default=(1, 1)\nThe lower and upper boundary of the range of n-values for different word n-grams or char n-grams to be extracted. All values \nof n such such that min_n <= n <= max_n will be used. For example an ngram_range of (1, 1) means only unigrams, (1, 2) means \nunigrams and bigrams, and (2, 2) means only bigrams. Only applies if analyzer is not callable.'

In [19]:
train = df['x'][ :900]
test = df['x'][900:]
x_train = vectorizer.fit_transform(train)

In [20]:
y_train = df['y'][ :900]

In [ ]:
label_classes=[]
for  key in y_train.vocabulary_.keys():
    label_classes.append(key)

In [23]:
from sklearn.preprocessing import MultiLabelBinarizer
mb = MultiLabelBinarizer()
Y = mb.fit_transform(y_train)

In [25]:
Y.shape

(900, 42)

In [27]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train,Y)

ValueError: y should be a 1d array, got an array of shape (900, 42) instead.